## Importing libraries 

In [1]:
import time
import statistics
import pandas as pd
import numpy as np
import copy as cp
import gensim.downloader
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from nltk.tokenize import word_tokenize
from sklearn.neighbors import NearestNeighbors

## Model importing

In [2]:
# Show all available models in gensim-data

print("Models:")
print(list(gensim.downloader.info()['models'].keys()),"\n")

model = gensim.downloader.load('glove-twitter-25')
# model.save("word2vec.model")

# print("Model download:")
# model = gensim.load('word2vec-google-news-300')

Models:
['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'] 



## Bases importing

In [3]:
qa_base = pd.read_csv('./insurance_qna_dataset.csv',sep='\t',index_col=0)
questions_number = qa_base.shape[0]

qm_base = pd.read_csv("Test questions dataset.csv")
test_questions_number = qm_base.shape[0]

# qa_base.head()
qm_base.head(21)

,original question,modified question – light,modified question – medium,modified question – heavy
0,How Many People Go Without Health Insurance?,How Many People Live Without Health Insurance?,How Many People Lack Health Coverage?,How Many Individuals Lack Good Health Coverage?
1,What Is The Purpose Of A Life Insurance Policy?,What Is The Point Of A Life Insurance Policy?,What Serves As A Life Insurance Policy's Objec...,What Do Life Insurance Policies Serve?
2,How Much Is A Typical Homeowners Insurance?,How Much Is An Average Homeowners Insurance?,How Much Does The Average Homeowners Insurance...,What Is the Average Cost Of Homeowners Insurance?
3,Is Disability Insurance Worth Having?,Is Disability Insurance Good?,Is Disability Insurance Worth Enough To Have It?,What Is The Value of Disability Insurance?
4,Is Vision Covered Under Medicare?,Is Vision Not Covered Under Medicare?,Does Medicare Cover Vision Care?,Is Medicare Going To Cover Vision Care?
5,Does Life Insurance Cover Both Spouses?,Does Life Insurance Cover Both Spouse?,Does Life Insurance Cover Both Partners?,Are Both Spouses Covered By Life Insurance?
6,Can Bad Credit Affect Car Insurance?,Can Bad Credit Affect Cars Insurance?,Can Bad Credits Affects Cars Insurance?,Can A Poor Credit History Affect Auto Insurance?
7,How Much Should I Get Life Insurance For?,How Much Should I I I Get Life Insurance For?,How Long Should I Get Life Insurance For?,How Much Life Insurance Should I Purchase?
8,Who Underwrites Hsbc Life Insurance?,Who Underwrite Hsbc Life Insurances?,Who Underwrites Hsbbcc Life Insurances?,Who Drafts The Hsbc Life Insurance Policy?
9,How Much Is Home Insurance Per Month?,How Much Is Home Insurance Per Months?,How Much Is House Insurance Per Month?,How Much Does Monthly Home Insurance Cost?


## Words tokenization

In [4]:
qa_base_tokenized = []
    
for i in range(questions_number):
    sentence = qa_base["Question"][i]
    token_words = word_tokenize(sentence)
    if token_words[-1] == '?':
        token_words.pop()
    token_words_lower = [word.lower() for word in token_words]
    qa_base_tokenized.append(token_words_lower)

oq_tokenized = []
qlm_tokenized = []
qmm_tokenized = []
qhm_tokenized = []

for i in range(test_questions_number):
    original_question = qm_base["original question"][i]
    light_modified_question = qm_base["modified question – light"][i]
    medium_modified_question = qm_base["modified question – medium"][i]
    heavy_modified_question = qm_base["modified question – heavy"][i]
    
    token_words_oq = word_tokenize(original_question)
    token_words_qlm = word_tokenize(light_modified_question)
    token_words_qmm = word_tokenize(medium_modified_question)
    token_words_qhm = word_tokenize(heavy_modified_question)
    
    token_words_oq.pop()
    token_words_qlm.pop()
    token_words_qmm.pop()
    token_words_qhm.pop()
    
    token_words_oq_lower = [word.lower() for word in token_words_oq]
    token_words_qlm_lower = [word.lower() for word in token_words_qlm]
    token_words_qmm_lower = [word.lower() for word in token_words_qmm]
    token_words_qhm_lower = [word.lower() for word in token_words_qhm]
    
    oq_tokenized.append(token_words_oq_lower)
    qlm_tokenized.append(token_words_qlm_lower)
    qmm_tokenized.append(token_words_qmm_lower)
    qhm_tokenized.append(token_words_qhm_lower)

## Words vectorization

In [5]:
def vectorizer(sentence):
    vectorized_sentence = []
    for i in range(len(sentence)):
        try:
            word = model[sentence[i]].tolist()
            vectorized_sentence.append(word)
        except:
            pass
    return vectorized_sentence
            

def token_sum(sentence):
    vector_length = len(sentence[0])
    sum_list = []
    for i in range(vector_length):
        counter = 0
        for word in sentence:
            counter += word[i]
        sum_list.append(counter)
    return sum_list
    
def token_mean(sentence):
    vector_length = len(sentence[0])
    mean_list = []
    for i in range(vector_length):
        local_mean_list = []
        for word in sentence:
            local_mean_list.append(word[i])
        mean_list.append(np.mean(local_mean_list))
    return mean_list

qa_base_vectorized_sum = []
qa_base_vectorized_mean = []

for i in range(questions_number):
    sentence = vectorizer(qa_base_tokenized[i])
    sentence_sum = token_sum(sentence)
    sentence_mean = token_mean(sentence)
    qa_base_vectorized_sum.append(sentence_sum)
    qa_base_vectorized_mean.append(sentence_mean)
    
qlm_vectorized_sum = []
qlm_vectorized_mean = []
qmm_vectorized_sum = []
qmm_vectorized_mean = []
qhm_vectorized_sum = []
qhm_vectorized_mean = []

for i in range(test_questions_number):
    qlm_sentence = vectorizer(qlm_tokenized[i])
    qmm_sentence = vectorizer(qmm_tokenized[i])
    qhm_sentence = vectorizer(qhm_tokenized[i])
    
    qlm_sentence_sum = token_sum(qlm_sentence)
    qmm_sentence_sum = token_sum(qmm_sentence)
    qhm_sentence_sum = token_sum(qhm_sentence)
    
    qlm_sentence_mean = token_mean(qlm_sentence)
    qmm_sentence_mean = token_mean(qmm_sentence)
    qhm_sentence_mean = token_mean(qhm_sentence)
    
    qlm_vectorized_sum.append(qlm_sentence_sum)
    qmm_vectorized_sum.append(qmm_sentence_sum)
    qhm_vectorized_sum.append(qhm_sentence_sum)
    
    qlm_vectorized_mean.append(qlm_sentence_mean)
    qmm_vectorized_mean.append(qmm_sentence_mean)
    qhm_vectorized_mean.append(qhm_sentence_mean)

## Nearest Neighbors space creation

In [7]:
nbrs_euclidean_sum = NearestNeighbors(n_neighbors=25000, metric="euclidean").fit(qa_base_vectorized_sum)
nbrs_manhattan_sum = NearestNeighbors(n_neighbors=25000, metric="manhattan").fit(qa_base_vectorized_sum)
nbrs_cosine_sum = NearestNeighbors(n_neighbors=25000, metric="cosine").fit(qa_base_vectorized_sum)

nbrs_euclidean_mean = NearestNeighbors(n_neighbors=25000, metric="euclidean").fit(qa_base_vectorized_mean)
nbrs_manhattan_mean = NearestNeighbors(n_neighbors=25000, metric="manhattan").fit(qa_base_vectorized_mean)
nbrs_cosine_mean = NearestNeighbors(n_neighbors=25000, metric="cosine").fit(qa_base_vectorized_mean)

## Recording results

In [8]:
results = pd.DataFrame(columns = ['arithmetics',
                                  'questions modification',
                                  'metric',
                                  'average sentence rank',
                                  'accuracy [%]',
                                  'average execution time [s]'])

## Lightly modified questions - sum arithmetics

In [9]:
euclidean_ranks = []
manhattan_ranks = []
cos_ranks = []

euclidean_target_counter = 0
manhattan_target_counter = 0
cos_target_counter = 0

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = np.reshape(qlm_vectorized_sum[i],(1,-1))
    
    distances_euclidean, indices_euclidean = nbrs_euclidean_sum.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan_sum.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine_sum.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank = nbrs_list_euclidean.index(original_question)+1
    manhattan_rank = nbrs_list_manhattan.index(original_question)+1
    cos_rank = nbrs_list_cosine.index(original_question)+1
    
    euclidean_ranks.append(euclidean_rank)
    manhattan_ranks.append(manhattan_rank)
    cos_ranks.append(cos_rank)
    
    if euclidean_rank <= 300:
        euclidean_target_counter += 1
    if manhattan_rank <= 300:
        manhattan_target_counter += 1
    if cos_rank <= 300:
        cos_target_counter += 1
    
end = time.time()
t = (end-start)/test_questions_number

print("Light modifications - euclidian ranks:", euclidean_ranks)
print("Light modifications - manhattan ranks:", manhattan_ranks)
print("Light modifications - cos ranks:", cos_ranks)

data = {'arithmetics':'sum',
         'questions modification':'light',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'accuracy [%]':round((euclidean_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'sum',
         'questions modification':'light',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'accuracy [%]':round((manhattan_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'sum',
         'questions modification':'light',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'accuracy [%]':round((cos_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Light modifications - euclidian ranks: [1, 1, 499, 514, 53, 2, 1, 1556, 365, 1, 3, 1, 4, 1, 1, 4258, 2, 1, 200, 1443, 12642]
Light modifications - manhattan ranks: [1, 1, 835, 173, 2, 6, 1, 280, 216, 1, 1, 1, 1, 1, 1, 43, 3, 1, 10, 2304, 7328]
Light modifications - cos ranks: [1, 3, 2026, 224, 1, 15, 1, 14, 1216, 1, 1, 1, 1, 1, 1, 8, 5, 1, 1, 1124, 5276]


/tmp/ipykernel_91909/2077333932.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/2077333932.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/2077333932.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## Lightly modified questions - mean arithmetics

In [10]:
euclidean_ranks = []
manhattan_ranks = []
cos_ranks = []

euclidean_target_counter = 0
manhattan_target_counter = 0
cos_target_counter = 0

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = np.reshape(qlm_vectorized_mean[i],(1,-1))
    
    distances_euclidean, indices_euclidean = nbrs_euclidean_mean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan_mean.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine_mean.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank = nbrs_list_euclidean.index(original_question)+1
    manhattan_rank = nbrs_list_manhattan.index(original_question)+1
    cos_rank = nbrs_list_cosine.index(original_question)+1
    
    euclidean_ranks.append(euclidean_rank)
    manhattan_ranks.append(manhattan_rank)
    cos_ranks.append(cos_rank)
    
    if euclidean_rank <= 300:
        euclidean_target_counter += 1
    if manhattan_rank <= 300:
        manhattan_target_counter += 1
    if cos_rank <= 300:
        cos_target_counter += 1
    
end = time.time()
t = (end-start)/test_questions_number

print("Light modifications - euclidian ranks:", euclidean_ranks)
print("Light modifications - manhattan ranks:", manhattan_ranks)
print("Light modifications - cos ranks:", cos_ranks)

data = {'arithmetics':'mean',
         'questions modification':'light',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'accuracy [%]':round((euclidean_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'mean',
         'questions modification':'light',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'accuracy [%]':round((manhattan_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'mean',
         'questions modification':'light',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'accuracy [%]':round((cos_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Light modifications - euclidian ranks: [1, 1, 1073, 192, 1, 1, 1, 8, 2594, 1, 1, 1, 1, 1, 1, 8, 5, 1, 3, 1106, 7756]
Light modifications - manhattan ranks: [1, 1, 1659, 196, 1, 6, 1, 11, 762, 1, 1, 1, 1, 1, 1, 5, 7, 1, 3, 2417, 6101]
Light modifications - cos ranks: [1, 3, 2026, 224, 1, 15, 1, 14, 1216, 1, 1, 1, 1, 1, 1, 8, 5, 1, 1, 1124, 5276]


/tmp/ipykernel_91909/205558446.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/205558446.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/205558446.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## Medium modified questions - sum arithmetics

In [11]:
euclidean_ranks = []
manhattan_ranks = []
cos_ranks = []

euclidean_target_counter = 0
manhattan_target_counter = 0
cos_target_counter = 0

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = np.reshape(qmm_vectorized_sum[i],(1,-1))
    
    distances_euclidean, indices_euclidean = nbrs_euclidean_sum.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan_sum.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine_sum.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank = nbrs_list_euclidean.index(original_question)+1
    manhattan_rank = nbrs_list_manhattan.index(original_question)+1
    cos_rank = nbrs_list_cosine.index(original_question)+1
    
    euclidean_ranks.append(euclidean_rank)
    manhattan_ranks.append(manhattan_rank)
    cos_ranks.append(cos_rank)
    
    if euclidean_rank <= 300:
        euclidean_target_counter += 1
    if manhattan_rank <= 300:
        manhattan_target_counter += 1
    if cos_rank <= 300:
        cos_target_counter += 1
    
end = time.time()
t = (end-start)/test_questions_number

print("Medium modifications - euclidian ranks:", euclidean_ranks)
print("Medium modifications - manhattan ranks:", manhattan_ranks)
print("Medium modifications - cos ranks:", cos_ranks)

data = {'arithmetics':'sum',
         'questions modification':'medium',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'accuracy [%]':round((euclidean_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'sum',
         'questions modification':'medium',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'accuracy [%]':round((manhattan_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'sum',
         'questions modification':'medium',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'accuracy [%]':round((cos_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Medium modifications - euclidian ranks: [2214, 838, 2966, 21698, 1332, 1, 231, 1, 307, 1, 4, 2091, 158, 1, 2, 7, 207, 45, 4809, 3859, 271]
Medium modifications - manhattan ranks: [221, 237, 4403, 12209, 2813, 1, 111, 1, 262, 1, 2, 561, 13, 1, 2, 5, 9, 63, 492, 4441, 342]
Medium modifications - cos ranks: [55, 108, 2227, 12212, 19326, 1, 57, 1, 5881, 1, 1, 690, 39, 1, 2, 5, 1, 172, 144, 3499, 1676]


/tmp/ipykernel_91909/4271705371.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/4271705371.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/4271705371.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## Medium modified questions - mean arithmetics

In [12]:
euclidean_ranks = []
manhattan_ranks = []
cos_ranks = []

euclidean_target_counter = 0
manhattan_target_counter = 0
cos_target_counter = 0

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = np.reshape(qmm_vectorized_mean[i],(1,-1))
    
    distances_euclidean, indices_euclidean = nbrs_euclidean_mean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan_mean.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine_mean.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank = nbrs_list_euclidean.index(original_question)+1
    manhattan_rank = nbrs_list_manhattan.index(original_question)+1
    cos_rank = nbrs_list_cosine.index(original_question)+1
    
    euclidean_ranks.append(euclidean_rank)
    manhattan_ranks.append(manhattan_rank)
    cos_ranks.append(cos_rank)
    
    if euclidean_rank <= 300:
        euclidean_target_counter += 1
    if manhattan_rank <= 300:
        manhattan_target_counter += 1
    if cos_rank <= 300:
        cos_target_counter += 1
    
end = time.time()
t = (end-start)/test_questions_number

print("Medium modifications - euclidian ranks:", euclidean_ranks)
print("Medium modifications - manhattan ranks:", manhattan_ranks)
print("Medium modifications - cos ranks:", cos_ranks)

data = {'arithmetics':'mean',
         'questions modification':'medium',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'accuracy [%]':round((euclidean_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'mean',
         'questions modification':'medium',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'accuracy [%]':round((manhattan_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'mean',
         'questions modification':'medium',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'accuracy [%]':round((cos_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Medium modifications - euclidian ranks: [108, 1426, 1717, 12109, 7923, 1, 214, 1, 1898, 1, 3, 639, 65, 17, 2, 5, 1, 71, 168, 3511, 1579]
Medium modifications - manhattan ranks: [123, 421, 2470, 11418, 13366, 1, 104, 1, 5857, 1, 3, 736, 11, 4, 2, 1, 1, 99, 145, 4556, 1806]
Medium modifications - cos ranks: [55, 108, 2227, 12212, 19326, 1, 57, 1, 5881, 1, 1, 690, 39, 1, 2, 5, 1, 172, 144, 3499, 1676]


/tmp/ipykernel_91909/3937760734.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/3937760734.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/3937760734.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## Heavily modified questions - sum arithmetics

In [13]:
euclidean_ranks = []
manhattan_ranks = []
cos_ranks = []

euclidean_target_counter = 0
manhattan_target_counter = 0
cos_target_counter = 0

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = np.reshape(qhm_vectorized_sum[i],(1,-1))
    
    distances_euclidean, indices_euclidean = nbrs_euclidean_sum.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan_sum.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine_sum.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank = nbrs_list_euclidean.index(original_question)+1
    manhattan_rank = nbrs_list_manhattan.index(original_question)+1
    cos_rank = nbrs_list_cosine.index(original_question)+1
    
    euclidean_ranks.append(euclidean_rank)
    manhattan_ranks.append(manhattan_rank)
    cos_ranks.append(cos_rank)
    
    if euclidean_rank <= 300:
        euclidean_target_counter += 1
    if manhattan_rank <= 300:
        manhattan_target_counter += 1
    if cos_rank <= 300:
        cos_target_counter += 1
    
end = time.time()
t = (end-start)/test_questions_number

print("Heavy modifications - euclidian ranks:", euclidean_ranks)
print("Heavy modifications - manhattan ranks:", manhattan_ranks)
print("Heavy modifications - cos ranks:", cos_ranks)

data = {'arithmetics':'sum',
         'questions modification':'heavy',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'accuracy [%]':round((euclidean_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'sum',
         'questions modification':'heavy',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'accuracy [%]':round((manhattan_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'sum',
         'questions modification':'heavy',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'accuracy [%]':round((cos_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Heavy modifications - euclidian ranks: [374, 24002, 5675, 16760, 19570, 101, 4851, 5829, 16064, 3223, 1813, 854, 182, 1, 3885, 15106, 1, 7376, 3396, 3423, 10887]
Heavy modifications - manhattan ranks: [134, 22004, 4396, 7475, 16636, 32, 2376, 898, 6635, 4035, 2103, 313, 210, 1, 4325, 3518, 1, 2026, 694, 2938, 4722]
Heavy modifications - cos ranks: [164, 6980, 5689, 5631, 22502, 10, 4324, 36, 6, 9222, 3184, 320, 48, 1, 3677, 10384, 1, 859, 115, 820, 2756]


/tmp/ipykernel_91909/4050413619.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/4050413619.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/4050413619.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## Heavily modified questions - mean arithmetics

In [14]:
euclidean_ranks = []
manhattan_ranks = []
cos_ranks = []

euclidean_target_counter = 0
manhattan_target_counter = 0
cos_target_counter = 0

start = time.time()

for i in range(test_questions_number):
    
    original_question = qm_base["original question"][i]
    modified_question = np.reshape(qhm_vectorized_mean[i],(1,-1))
    
    distances_euclidean, indices_euclidean = nbrs_euclidean_mean.kneighbors(modified_question)
    distances_manhattan, indices_manhattan = nbrs_manhattan_mean.kneighbors(modified_question)
    distances_cosine, indices_cosine = nbrs_cosine_mean.kneighbors(modified_question)
    
    nbrs_list_euclidean = qa_base["Question"].iloc[indices_euclidean[0]].tolist()
    nbrs_list_manhattan = qa_base["Question"].iloc[indices_manhattan[0]].tolist()
    nbrs_list_cosine = qa_base["Question"].iloc[indices_cosine[0]].tolist()
    
    euclidean_rank = nbrs_list_euclidean.index(original_question)+1
    manhattan_rank = nbrs_list_manhattan.index(original_question)+1
    cos_rank = nbrs_list_cosine.index(original_question)+1
    
    euclidean_ranks.append(euclidean_rank)
    manhattan_ranks.append(manhattan_rank)
    cos_ranks.append(cos_rank)
    
    if euclidean_rank <= 300:
        euclidean_target_counter += 1
    if manhattan_rank <= 300:
        manhattan_target_counter += 1
    if cos_rank <= 300:
        cos_target_counter += 1
    
end = time.time()
t = (end-start)/test_questions_number

print("Heavy modifications - euclidian ranks:", euclidean_ranks)
print("Heavy modifications - manhattan ranks:", manhattan_ranks)
print("Heavy modifications - cos ranks:", cos_ranks)

data = {'arithmetics':'mean',
         'questions modification':'heavy',
         'metric':'euclidean',
         'average sentence rank':round(np.mean(euclidean_rank)),
         'accuracy [%]':round((euclidean_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'mean',
         'questions modification':'heavy',
         'metric':'manhattan',
         'average sentence rank':round(np.mean(manhattan_rank)),
         'accuracy [%]':round((manhattan_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

data = {'arithmetics':'mean',
         'questions modification':'heavy',
         'metric':'cosine',
         'average sentence rank':round(np.mean(cos_rank)),
         'accuracy [%]':round((cos_target_counter/test_questions_number)*100,2),
         'average execution time [s]':round(t,4)}

results = results.append(data, ignore_index=True)

Heavy modifications - euclidian ranks: [842, 11985, 5075, 5255, 22023, 3, 2173, 294, 1, 7163, 2671, 248, 219, 2, 3740, 9397, 1, 58, 265, 539, 1767]
Heavy modifications - manhattan ranks: [328, 14276, 3538, 5421, 23016, 5, 1712, 181, 4, 7583, 2784, 120, 277, 1, 4808, 2794, 1, 69, 252, 885, 804]
Heavy modifications - cos ranks: [164, 6980, 5689, 5631, 22502, 10, 4324, 36, 6, 9222, 3184, 320, 48, 1, 3677, 10384, 1, 859, 115, 820, 2756]


/tmp/ipykernel_91909/413118105.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/413118105.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)
/tmp/ipykernel_91909/413118105.py:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(data, ignore_index=True)


## View of results

In [15]:
results.to_csv("results - word vectorization.csv",index=False)

results.head(60)

,arithmetics,questions modification,metric,average sentence rank,accuracy [%],average execution time [s]
0,sum,light,euclidean,12642,66.67,0.035
1,sum,light,manhattan,7328,85.71,0.035
2,sum,light,cosine,5276,80.95,0.035
3,mean,light,euclidean,7756,80.95,0.0302
4,mean,light,manhattan,6101,80.95,0.0302
5,mean,light,cosine,5276,80.95,0.0302
6,sum,medium,euclidean,271,57.14,0.0298
7,sum,medium,manhattan,342,66.67,0.0298
8,sum,medium,cosine,1676,66.67,0.0298
9,mean,medium,euclidean,1579,61.9,0.0309
